# Math Inverse Kinematics

In [1]:
# Import essential modules
import numpy as np
from forward_kinematics import forward_kinematics

## Create task what we'll solve

In [2]:
fT1=0
fT2=25
fT3=82
fT4=49
fT2+fT3+fT4

156

In [3]:
def cos(t):
    return np.cos(np.deg2rad(t))
def sin(t):
    return np.sin(np.deg2rad(t))

In [4]:
fk = forward_kinematics(fT1,fT2,fT3,fT4)[:3,3]
fk

array([200.,  -0.,   3.])

In [5]:
dx=fk[0]
dy=fk[1]
dz=fk[2]

In [6]:
dx,dy,dz

(200.0, -0.0, 3.0)

## Use math to solve IK

### Theta 1

In [7]:
T1= np.arctan(dy/dx)
T1=np.round(np.degrees(-T1))
T1

0.0

### Theta 3

In [7]:
l1=104.
l2=89.
l3=175.
a1=95.
fi=fT2+fT3+fT4
eT2=(90-fT2)
eT3=(90-fT3)+eT2
eT4=(90-fT4)+eT3
#dz=dz-95

In [9]:
eT2,eT3,eT4

(65, 73, 114)

In [10]:
# Calculate r
r = np.sqrt(dx**2+dy**2)
r

200.0

In [11]:
# Calculate r1 if we know fT2
r1 = cos(eT2)*l1
r1

43.95229922103274

In [12]:
# Calcualte r2
r2 = r-r1
r2

156.04770077896725

In [13]:
# Find t
t = sin(eT2)*l1
t

94.2560098518116

In [14]:
t1=t-(dz-95)
t1

186.2560098518116

In [15]:
alpha1=np.arctan(r2/t1)
alpha1=np.degrees(alpha1)
alpha1

39.95671175835587

In [16]:
b=np.sqrt(t1**2+r2**2)
b

242.9859792751842

In [17]:
operation1=(l2**2+b**2-l3**2)/(2*l2*b)
operation1*1000

840.1600523081219

In [18]:
alpha2=np.arccos((l2**2+b**2-l3**2)/(2*l2*b))
alpha2=np.degrees(alpha2)
alpha2

32.84297539052342

In [19]:
T3=180-(alpha1+alpha2+fT2)
T3=np.round(T3)
T3

82.0

In [20]:
if(t1>0):
    T3=180-(alpha1+alpha2+fT2)
    T3=np.round(T3)
    print(T3)
else:
    T3=alpha1+alpha2+fT2
    T3=abs(np.round(T3))
    print(T3)

82.0


### Theta 4

In [21]:
T4=np.arccos((l2**2+l3**2-b**2)/(2*l2*l3))
T4=np.degrees(T4)
T4

131.1462640094422

In [22]:
T4=180-T4
T4=np.round(T4)
T4

49.0

In [23]:
print(f"Pred T1:{T1}, T2:{fT2}, T3:{T3} ,T4:{T4}")
print(f"True: T1:{fT1}, T2:{fT2}, T3:{fT3} ,T4:{fT4}")
print(f"Jesu li isti: {[T1,fT2,T3,T4]==[fT1,fT2,fT3,fT4]}")

Pred T1:0.0, T2:25, T3:82.0 ,T4:49.0
True: T1:0, T2:25, T3:82 ,T4:49
Jesu li isti: True


## Make functons for IK

In [7]:
from sklearn.metrics import mean_squared_error

def finder(fk, i):
    dx,dy,dz=fk
    eT2=(90-i)
    l1=104.
    l2=89.
    l3=175.
    r = np.sqrt(dx**2+dy**2)
    r1 = cos(eT2)*l1
    r2 = r-r1
    t = sin(eT2)*l1
    t1=t-(dz-95)
    if(t1!=0):        
        alpha1=np.arctan(r2/t1)
        alpha1=np.degrees(alpha1)
    else:
        alpha1=0
    b=np.sqrt(t1**2+r2**2)
    operation1=(l2**2+b**2-l3**2)/(2*l2*b)
    operation2=(l2**2+l3**2-b**2)/(2*l2*l3)
    if(int(operation1*100)>100 or int(operation1*100)<-100):
        raise Exception() 
    if(int(operation1*100)==100 or int(operation1*100)==-100):
        operation1=np.round(operation1)
        
    if(int(operation2*100)>100 or int(operation2*100)<-100):
        raise Exception() 
    if(int(operation2*100)==100 or int(operation2*100)==-100):
        operation2=np.round(operation2)

    alpha2=np.arccos(operation1)
    alpha2=np.degrees(alpha2)
    if(t1>0):
        T3=180-(alpha1+alpha2+i)
        T3=np.round(T3)
    else:
        T3=alpha1+alpha2+i
        T3=abs(np.round(T3))
    T4=np.arccos(operation2)

    T4=np.degrees(T4)
    T4=180-T4
    T4=np.round(T4)

    if T3>90 or T4>90:
        raise Exception()
    return T3, T4

def find_T2(T1,fk_true):

    mse={}

    for i in range(91):
        try:
            T3, T4 = finder(fk_true, i)
            fk =forward_kinematics(T1,i,T3,T4)[:3,3]
            if (np.array_equal(fk_true,fk) and np.isclose(180,(i+T3+T4),atol=5)):
                return i, T3, T4
            mse[i]=[mean_absolute_error(np.append(fk_true,180),np.append(fk,i+T3+T4)),T3,T4]
        except:
            pass
    else:
        T2 =min(mse,key=mse.get)
        T3, T4 = mse[T2][1:]
        return T2,T3,T4
def calculate_inverse_kinematics(fk):
    dx,dy,dz=fk
    #if(np.sqrt(dx**2+dy**2)-dz<64):
       # raise Exception("It was entered lower value than arm can reach")
    # Find T1
    if(dx==0.):
        T1=90
    else:
        T1= np.arctan(dy/dx)
        T1=np.degrees(-T1)
        T1=np.round(T1)
    #Find T3
    T2, T3, T4=find_T2(T1,fk)
    eT2=(90-T2)
    eT3=(90-T3)+eT2
    eT4=(90-T4)+eT3
   # print(T2+T3+T4)
    return float(T1), float(T2), float(T3), float(T4)

In [67]:
kT1=-74
kT2=31
kT3=19
kT4=71
fk=forward_kinematics(kT1,kT2,kT3,kT4)[:3,3]
print(fk, [kT1,kT2,kT3,kT4])

[ 75. 261. 151.] [-74, 31, 19, 71]


In [68]:
result=calculate_inverse_kineamtics([200,-50,20])
result

ValueError: min() arg is an empty sequence

In [ ]:
forward_kinematics(result[0],result[1],result[2],result[3])[:3,3], fk

In [28]:
y_true=[]
y_pred=[]
for i in range(5000):
    try:
        T1=np.random.randint(-90,91)
        T2=np.random.randint(0,91)
        T3=np.random.randint(0,91)
        T4=np.random.randint(0,91)       
        fk=forward_kinematics(T1,T2,T3,T4)[:3,3]
        result=calculate_inverse_kineamtics(fk)
        y_true.append(fk)
        y_pred.append(forward_kinematics(result[0],result[1],result[2],result[3])[:3,3])
    except:
        pass

In [29]:

from sklearn.metrics import mean_absolute_error 
mean_absolute_error(y_true,y_pred)

0.0

In [30]:
T1=np.random.randint(-90,91)
T2=np.random.randint(0,91)
T3=np.random.randint(0,91)
T4=np.random.randint(0,91)       
fk=forward_kinematics(T1,T2,T3,T4)[:3,3]
result=calculate_inverse_kineamtics(fk)
print(f"Pred T1:{result[0]}, T2:{result[1]}, T3:{result[2]} ,T4:{result[3]}")
print(f"True: T1:{T1}, T2:{T2}, T3:{T3} ,T4:{T4}")
fk_pred=forward_kinematics(result[0],result[1],result[2],result[3])[:3,3]
print(f"Jesu li isti: {np.array_equal(fk_pred,fk)}")
fk_pred, fk


Pred T1:40.0, T2:26, T3:33.0 ,T4:23.0
True: T1:40, T2:23, T3:46 ,T4:9
Jesu li isti: True


(array([ 226., -190.,  259.]), array([ 226., -190.,  259.]))

In [ ]:
k=0
for i in range(50000):
    try:
        T1=np.random.randint(-90,91)
        T2=np.random.randint(0,91)
        T3=np.random.randint(0,91)
        T4=np.random.randint(0,91)       
        fk=forward_kinematics(T1,T2,T3,T4)[:3,3]
        result=calculate_inverse_kineamtics(fk)
        fk_pred=forward_kinematics(result[0],result[1],result[2],result[3])[:3,3]
        if(np.array_equal(fk_pred,fk)==False):
            #print("false")
            k+=1
    except:
        pass
print(k)